In [3]:
# https://gist.github.com/EderSantana/c7222daa328f0e885093 much code and inspiration from here, il iterate from this starting point

# Normal, choose max reward action
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.advanced_activations import PReLU
import numpy as np
import gym

EPSILON = 0.2
NUM_ACTIONS = 2
STATE_SPACE = 4
MAX_MEMORY = 160
EPOCH = 1000 
BATCH_SIZE=8


def weighted_choice(weights):
    totals = np.cumsum(weights)
    norm = totals[-1]
    throw = np.random.rand()*norm
    return np.searchsorted(totals, throw)


class ExperienceReplay(object):
    def __init__(self, discount=.9):
        self.memory = []
        self.discount = discount

    def remember(self, states, game_over):
        self.memory.append([states, game_over])
        if len(self.memory) > MAX_MEMORY:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        inputs = np.zeros((min(len(self.memory), batch_size), STATE_SPACE))
        targets = np.zeros((inputs.shape[0], NUM_ACTIONS))
        for i, idx in enumerate(np.random.randint(0, len(self.memory),
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            targets[i] = model.predict(np.reshape(state_t, (1, state_t.shape[0])))[0]
            Q_sa = np.max(model.predict(np.reshape(state_tp1, (1, state_tp1.shape[0])))[0])
            if game_over: 
                targets[i, action_t] = reward_t
            else:
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets


def ker_model(input_size, output_size):
    model = Sequential()
    model.add(Dense(256, input_shape=(input_size,)))
    model.add(PReLU())
    model.add(Dense(256))
    model.add(PReLU())
    model.add(Dense(output_size))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    return model

env = gym.make('CartPole-v0')
exp_replay = ExperienceReplay()
model = ker_model(STATE_SPACE, NUM_ACTIONS)
env.render()
for e in range(EPOCH):
    win_cnt = 0
    loss = 0.
    input_t = env.reset()
    game_over = False

    while not game_over:
        input_tm1 = input_t
        # get next action
        if e < 100:
            if np.random.rand() <= EPSILON:
                action = np.random.randint(0, NUM_ACTIONS, size=1)[0]
            else:
                
                q = model.predict(np.reshape(input_tm1, (1, input_tm1.shape[0])))
                action = np.argmax(q[0])
        else:
            q = model.predict(np.reshape(input_tm1, (1, input_tm1.shape[0])))
            action = np.argmax(q[0])
        
        # Cartpole goes from -1 to 1
        # action -= 1
        
        # apply action, get rewards and new state
        input_t, reward, game_over, info = env.step(action)
        win_cnt += reward

        # store experience
        exp_replay.remember([input_tm1, action, reward, input_t], game_over)

        # adapt model
        inputs, targets = exp_replay.get_batch(model, batch_size=BATCH_SIZE)
        loss += model.train_on_batch(inputs, targets)
    print("Epoch {:03d}/999 | Loss {:.4f} | Win count {}".format(e, loss, win_cnt))

[2017-08-06 20:51:06,796] Making new env: CartPole-v0


C:\Python35_64\lib\site-packages\cntk\core.py:351: UserWarning: your data is of type "float64", but your input variable (uid "Input2568") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
C:\Python35_64\lib\site-packages\cntk\core.py:351: UserWarning: your data is of type "float64", but your input variable (uid "Input2657") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               1280      
_________________________________________________________________
p_re_lu_7 (PReLU)            (None, 256)               256       
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
p_re_lu_8 (PReLU)            (None, 256)               256       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 514       
Total params: 68,098
Trainable params: 68,098
Non-trainable params: 0
_________________________________________________________________


Epoch 000/999 | Loss 9.6624 | Win count 12.0


Epoch 001/999 | Loss 17.7941 | Win count 11.0


...

Epoch 129/999 | Loss 8.1909 | Win count 140.0


Epoch 130/999 | Loss 10.9653 | Win count 100.0


Epoch 131/999 | Loss 28.2814 | Win count 155.0


Epoch 132/999 | Loss 36.7819 | Win count 200.0


SystemError: <built-in function Trainer_train_minibatch> returned a result with an error set